# **Import**

In [ ]:
import sys
sys.path.append('..')

from pydantic import ValidationError
from botocore.exceptions import ClientError
from app.sync_storage import documents_service, create_buckets
from app.schemas import UploadFileRequest, DownloadFileRequest, DeleteFileRequest, GetFileRequest

# **Initialize**

In [ ]:
create_buckets()

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'Tarski1948.pdf' instead of 'assets/Tarski1948.pdf')
%cd assets/
%ls

# **Test `FileService`**

**Note**: Both `documents_service` and `images_service` are instances of the same `FileService` class and share identical internal logic — differing only in their target bucket name. Therefore, it is sufficient to test the method on one service instance; successful behavior implies equivalent correctness for the other.

## **I. Test `upload()` method**

In [ ]:
def test_upload_file(storage_key: str, file_path: str):
    try:
        request = UploadFileRequest(storage_key=storage_key, file_path=file_path)
        documents_service.upload(request)
        print(f"File '{storage_key}' uploaded successfully to bucket '{documents_service.bucket_name}'.")
        return True
        
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during upload ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during upload: {e}")
        return False
    
test_upload_file(storage_key="tarski-logic-book", file_path="Tarski1948.pdf")

In [ ]:
# Trying to upload a non-existent file
test_upload_file(storage_key="tarski-logic-book", file_path="Tarski1948")

## **II. Test `download()` method**

In [ ]:
def test_download_file(storage_key: str, file_path: str):
    try:
        request = DownloadFileRequest(storage_key=storage_key, file_path=file_path)
        documents_service.download(request)
        print(f"File '{storage_key}' downloaded successfully to '{file_path}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during download ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during download: {e}")
        return False
    
test_download_file(storage_key = "tarski-logic-book", file_path = "export.pdf")

In [ ]:
# Trying to download a non-existent file
test_download_file(storage_key = "hello-there", file_path = "export.pdf")

## **III. Test `get()` method**

In [ ]:
def test_get_file_metadata(storage_key: str):
    try:
        request = GetFileRequest(storage_key=storage_key)
        metadata = documents_service.get(request)
        print(f"Metadata retrieved for '{storage_key}':")
        print(f"Size: {metadata['ContentLength']} bytes")
        print(f"LastModified: {metadata['LastModified']}")
        print(f"ETag: {metadata['ETag']}")
        return metadata

    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during retrieval ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during retrieval: {e}")
        return False
    
test_get_file_metadata(storage_key = "tarski-logic-book")

## **IV. Test `get_all()` method**

In [ ]:
# Create a helper record to ensure get_all returns all files, not just one
test_upload_file(storage_key="kant-philosophy-book", file_path="critique-of-pure-reason.pdf")

In [ ]:
def test_list_files(max_keys: int = 10):
    try:
        objects = documents_service.get_all(max_keys=max_keys)
        if objects:
            print(f"Found {len(objects)} objects in bucket '{documents_service.bucket_name}':")
        else:
            print(f"Bucket '{documents_service.bucket_name}' is empty.")
        return objects
    
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during retrieval ({error_code}): {e}")
        return []
        
    except Exception as e:
        print(f"Unexpected error during retrieval: {e}")
        return []
    
test_list_files()

## **V. Test `delete()` method**

In [ ]:
def test_delete_file(storage_key: str):
    try:
        request = DeleteFileRequest(storage_key=storage_key)
        documents_service.delete(request)
        print(f"File '{storage_key}' deleted (or did not exist) from bucket '{documents_service.bucket_name}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False

    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during delete ({error_code}): {e}")
        return False
    
    except Exception as e:
        print(f"Unexpected error during delete: {e}")
        return False
    
test_delete_file(storage_key = "tarski-logic-book")

In [ ]:
# Trying to delete a non-existent file
test_delete_file(storage_key = "hello-there")

In [ ]:
# Trying to delete file with invalid name
test_delete_file(storage_key = " dthrjjf   ")